# Day 4: Syntax and Parsing

In [13]:
%matplotlib inline

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import sys
sys.path.append('/Users/davidbuchacaprats/Dropbox/lxmls2015/lxmls-toolkit-student')
import lxmls
import scipy
path_inside_lxmls_toolkit_student = "/Users/davidbuchacaprats/Dropbox/lxmls2015/lxmls-toolkit-student"

## Basic definitions for the session

### Definition: Context-free grammar 

A context-free grammar (CFG) is a tuple $ G= <\mathcal{N}, \mathcal{T}, \mathcal{R}, \mathcal{S}>$ where:


- N is a finite set of non-terminal symbols. Elements of N are denoted by upper case letters (A, B, C, ...). Each non-terminal symbol is a syntactic category: it represents a different type of phrase or clause in the sentence.

 - T is a finite set of terminal symbols (disjoint from N). Elements of $\mathcal{T}$ are denoted by lower case letters $(a, b, c, . . .)$. Each terminal symbol is a surface word: terminal symbols make up the actual content of sentences. The set $\mathcal{T}$ is called the alphabet of the language defined by the grammar $\mathcal{G}$.

- $\mathcal{R}$ is a set of 
production rules, i.e., a finite relation 
from $\mathcal{N}$ to $(\mathcal{N} \cup \mathcal{T})^*$. 
$G$ is said to be in Chomsky normal form (CNF) if production rules in $\mathcal{R}$ are either of the form 
$A \rightarrow B C$ or $A \rightarrow a$.

 - $\mathcal{S}$ is a "start symbol", used to represent the whole sentence. It must be an element of $\mathcal{N}$.

Any CFG can be transformed to be in CNF without loosing any expressive power in terms of the language it generates. Hence, we henceforth assume that G is in CNF without loss of generality.

Any CFG can be transformed to be in CNF without loosing any expressive power in terms of the language it generates. Hence, we henceforth assume that G is in CNF without loss of generality.

# CKY Algorithm: Step by Step

The CKY algorithm is a dynamic programming algorithm, much like Viterbi. The general idea is to build up on smaller, simpler things you have computed in previous iterations.

Viterby is focused on building sequences (of labels) by computing the highest scoring sequence of length $n$ from sequences of length $(n-1)$. CKY expands this idea to graphs, which can be thought as higher dimension sequences. In particular, trees spanning over $n$ words are built from trees spanning up to $(n-1)$ words.

In the next section, we're going to do some CKY iterations on a real sequence, to get a more concrete sense of how the algorithm works.

## CKY Algorithm recap


** WARNING ** Typo on the algorithm. It is delta(k,j+i-**1**,Y), not delta(k,j+i-**2**,Y). It is the only way to access elements that have been previously computed.


## Demo on concrete example

Inputs

* probabilities for production rules $\theta_{Z\rightarrow XY}$ and $\theta_{Z \rightarrow w}$
* sentence $s=$ _She enjoys the Summer School._

Notice that

* the length of the sequence is $N = 5$
* the terminal symbols that comprise the sentence are $w_1=$ _She_, $w_2=$ _enjoys_, $w_3=$ _the_, $w_4=$ _Summer_, $w_5=$ _School_

Variables to fill in:

* $\delta$, shape $(N\times N \times \textrm{num_symbols})$: partial probabilities
* $\psi$, shape $(N\times N \times \textrm{num_symbols})$: backtrack pointers

### Initialization

For all $i=1,...,N$, for all terminal symbols $Z\in \mathcal{N}$ for which there is a rule $Z \rightarrow w_i$, that is, for all $Z$ that may originate word $w_i$:

$$\delta(i, i, Z) = \theta_{Z\rightarrow w_i}$$

An example may clarify: assume for simplicity that $\theta_{VP \rightarrow w}$ is zero for any word except _enjoys_. After initialization,

$$
\delta(:\,, :\,, VP) = 
\begin{matrix}
0 & ? & ? & ? & ? \\
  & \theta_{VP \rightarrow enjoys} & ? & ? & ? \\
  &   & 0 & ? & ? \\
  &   &   & 0 & ? \\
  &   &   &   & 0
\end{matrix}
$$

the $?$'s will be filled in during the Induction stage.

### Induction

We now proceed to do some iterations on the Induction stage's double `for` loop:

#### Spans with length $i=2$ 

The first iteration on the outer loop considers subtrees spanning over pairs of words. $j$ determines which pair of words we are currently spanning over: for instance, $j=2$ corresponds to spanning over words $j=2$ and $j+i-1=3$.

For each $Z \in \mathcal{N}$, we want to find the most likely symbols $X, Y \in \mathcal{N}$ that can produce two words spanned by the subtree. To do this, we maximize over all $X,Y$ that $Z$ can generate.

Notice that we also maximize over $k$, which determines where the subtree splits. For $i=2$, there is only one value of $k$ per value of $j$. This is easy to understand: trees spanning over two words can only split in one way.

The table below summarizes the optimizations for all values of $j$. Do not forget that we are also optimizing over $X$ and $Y$.

|$\ i\ $|$\ j\ $|$\ k\ $|possible splits to maximize over|probability to fill with the max|
|--|--|
|2|1|2|$\delta(1,1,X)\cdot\delta(2,2,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(1,2,Z)$|
|2|2|3|$\delta(2,2,X)\cdot\delta(3,3,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(2,3,Z)$|
|2|3|4|$\delta(3,3,X)\cdot\delta(4,4,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(3,4,Z)$|
|2|4|5|$\delta(4,4,X)\cdot\delta(5,5,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(4,5,Z)$|

The figure below illustrates the subtree for $i=2$, $j=1$. There is only one way to split the tree ($k=2$), so the maximization is done over $X$ and $Y$.

<img src="./images_for_notebooks/day_4/cky_diagrams_i2.png" width=100>

Notice that the function we are optimizing are computed using elements of $\delta$ which have been defined during initialization: the main diagonals. After completing the loop for $i=2$ and all values of $j$, the deltas will have a few more values filled in. For instance:

$$
\delta(:\,, :\,, VP) = 
\begin{matrix}
0 & \blacklozenge  & ? & ? & ? \\
  & \theta_{VP \rightarrow enjoys} & \blacklozenge  & ? & ? \\
  &   & 0 & \blacklozenge  & ? \\
  &   &   & 0 & \blacklozenge  \\
  &   &   &   & 0
\end{matrix}
$$

where $\blacklozenge $ represents some numerical value.

#### Spans with length $i=3$

We now proceed to span over larger sequences. When spanning over more than 2 words, there are various possibilities for how the tree splits into subtrees (see figure below). This corresponds to having to optimize over $k$ in addition to $X$, and $Y$.  

|$\ i\ $|$\ j\ $|$\ k\ $|possible splits to maximize over|probability to fill with the max|
|--|--|
|3|1|2 or 3|$\delta(1,1,X)\cdot\delta(2,3,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(1,2,X)\cdot\delta(3,3,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(1,3,Z)$|
|3|2|3 or 4|$\delta(2,2,X)\cdot\delta(2,3,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(2,3,X)\cdot\delta(4,4,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(2,4,Z)$|
|3|3|4 or 5|$\delta(3,3,X)\cdot\delta(2,3,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(3,4,X)\cdot\delta(5,5,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(3,5,Z)$|

The figure below illustrates the subtree for $i=3$, $j=1$. There are two ways to split the tree ($k=2$ and $k=3$).

<img src="./images_for_notebooks/day_4/cky_diagrams_i3.png" width=300>

Again, notice we only need elements of $\delta$ that have already been computed.

#### Spans with length $i=4$

The procedure is similar to that of previous span lengths. The wider the span, the more possibilities there are for splitting the tree.

|$\ i\ $|$\ j\ $|$\ k\ $|possible splits to maximize over|probability to fill with the max|
|--|--|
|4|1|2, 3 or 4|$\delta(1,1,X)\cdot\delta(2,4,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(1,2,X)\cdot\delta(3,4,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(1,3,X)\cdot\delta(4,4,Y)\cdot\theta_{Z\rightarrow XY}$ |$\delta(1,4,Z)$
|4|2|3, 4 or 5|$\delta(2,2,X)\cdot\delta(2,5,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(2,3,X)\cdot\delta(4,5,Y)\cdot\theta_{Z\rightarrow XY}$ or $\delta(2,4,X)\cdot\delta(5,5,Y)\cdot\theta_{Z\rightarrow XY}$|$\delta(2,5,Z)$|

The figure below illustrates the subtree for $i=3$, $j=1$. There are three ways to split the tree ($k=2$, $k=3$ and $k=4$), which means that one has to maximize over $k$, $X$, and $Y$. Notice that we are not concerned how a set of three words (for instance, _She enjoys the_ on the far right) is spanned; the best subtree for spanning those words has been determined in the previous outer loop.

<img src="./images_for_notebooks/day_4/cky_diagrams_i4.png" width=600>

The last loop, $i=5$, corresponds to finding the tree for the whole sentence.

In the following exercises, you will see these steps take place in a live demo.

## Exercise 4.1

In this simple exercise, you will see the CKY algorithm in action. 

There is a Javascript applet that illustrates how CKY works (in its non-probabilistic form). Go to http://lxmls.it.pt/2015/cky.html, and observe carefully the several steps taken by the algorithm. Write down a small grammar in CNF that yields multiple parses for the ambiguous sentence The man saw the boy in the park with a telescope, and run the demo for this particular sentence. 

What would happen in the probabilistic form of CKY?

### Exercise 4.2

This exercise will show you that real-world sentences can have complicated syntactic structures. There is a parse tree visualizer in ```http://www.ark.cs.cmu.edu/treeviz/```. Go to your local data/treebanks folder and open the file PTB excerpt.txt. Copy a few trees from the file, one at a time, and examine their parse trees in the visualizer.

## Exercise 4.3

 In this exercise you are going to experiment with arc-factored non-projective dependency parsers.
The CoNLL-X and CoNLL 2008 shared task datasets (Buchholz and Marsi, 2006; Surdeanu et al., 2008) contain dependency treebanks for 14 languages. In this lab, we are going to experiment with the Portuguese and English datasets.
We preprocessed those datasets to exclude all sentences with more than 15 words; this yielded the files:

- data/deppars/portuguese train.conll,
- data/deppars/portuguese test.conll,
- data/deppars/english train.conll,
- data/deppars/english test.conll.

1) After importing all the necessary libraries, load the Portuguese dataset:

In [32]:
import lxmls.parsing.dependency_parser as depp
dp = depp.DependencyParser()
dp.read_data("portuguese")

Number of sentences: 3029
Number of tokens: 25015
Number of words: 7621
Number of pos: 16
Number of features: 142


Observe the statistics which are shown. How many features are there in total?

2) We will now have a close look on the features that can be used in the parser. Examine the file:

            ```lxmls/parsing/dependency features.py.```
    

The following method takes a sentence and computes a 
vector of features for each possible arc

```<h, m>```:


    def create_arc_features(self, instance, h, m, add=False):
        '''Creates features for arc h-->m.'''

We grouped the features in several subsets, so that we can conduct some ablation experiments:

- Basic features that look only at the parts-of-speech of the words that can be connected by an arc

- Lexical features that also look at these words themselves;

- Distance features that look at the length and direction of the dependency link (i.e., distance between the two words);

- Contextual features that look at the context (part-of-speech tags) of the words surrounding h and m.

In the default configuration, only the basic features are enabled. The total number of features is the quantity observed in the previous question. With this configuration, train the parser by running 10 epochs of the structured perceptron algorithm:

In [31]:
import lxmls.parsing.dependency_parser as depp
dp = depp.DependencyParser()
dp.read_data("portuguese")
dp.train_perceptron(10)

Number of sentences: 3029
Number of tokens: 25015
Number of words: 7621
Number of pos: 16
Number of features: 142
Epoch 1
Training accuracy: 0.497432605905
Epoch 2
Training accuracy: 0.499144201968
Epoch 3
Training accuracy: 0.498217087434
Epoch 4
Training accuracy: 0.50053487377
Epoch 5
Training accuracy: 0.501818570817
Epoch 6
Training accuracy: 0.498538011696
Epoch 7
Training accuracy: 0.500962772786
Epoch 8
Training accuracy: 0.500285266011
Epoch 9
Training accuracy: 0.499286834974
Epoch 10
Training accuracy: 0.500035658251


In [39]:
dp.test

<bound method DependencyParser.test of <lxmls.parsing.dependency_parser.DependencyParser instance at 0x10eaf9950>>

What is the accuracy obtained in the test set? (Note: the shown accuracy is the fraction of words whose parent was correctly predicted.)

3) Repeat the previous exercise by subsequently enabling the lexical, distance and contextual features:

In [10]:
dp.features.use_lexical = True 
dp.read_data("portuguese") 
dp.train_perceptron(10) 
dp.test()

Number of sentences: 3029
Number of tokens: 25015
Number of words: 7621
Number of pos: 16
Number of features: 46216
Epoch 1
Training accuracy: 0.531914134931
Epoch 2
Training accuracy: 0.641135358722
Epoch 3
Training accuracy: 0.722864070746
Epoch 4
Training accuracy: 0.784695478534
Epoch 5
Training accuracy: 0.820425046356
Epoch 6
Training accuracy: 0.851911282271
Epoch 7
Training accuracy: 0.873876765083
Epoch 8
Training accuracy: 0.890850092711
Epoch 9
Training accuracy: 0.897054628441
Epoch 10
Training accuracy: 0.907466837826
Test accuracy (109 test instances): 0.57662835249


In [ ]:
dp.features.use_distance = True
dp.read_data("portuguese") 
dp.train_perceptron(10) 
dp.test()

In [ ]:
dp.features.use_contextual = True 
dp.read_data("portuguese") 
dp.train_perceptron(10)
dp.test()

For each configuration, write down the number of features and test set accuracies. Observe the improvements obtained when more features were added.
Feel free to engineer new features!

4) Which of the three important inference tasks discussed above (computing the most likely tree, computing the parti- tion function, and computing the marginals) need to be performed in the structured perceptron algorithm? What about a maximum entropy classifier, with stochastic gradient descent? Check your answers by looking at the fol- lowing two methods in code/dependency parser.py:

    def train_perceptron(self, n_epochs): ...
   
    def train_crf_sgd(self, n_epochs, sigma, eta0 = 0.001): ...

Repeat the last exercise by training a maximum entropy classifier, with stochastic gradient descent, using l = 0.01 and a initial stepsize of $\eta_0$ = 0.1:

In [ ]:
dp.train_crf_sgd(10, 0.01, 0.1)
dp.test()

Compare the results with those obtained by the perceptron algorithm.

5) Train a parser for English using your favourite learning algorithm:

In [ ]:
dp.read_data("english")
dp.train_perceptron(10)
dp.test()

The predicted trees are placed in the file ```data/deppars/english_test.conll.pred```.
To get a sense of which errors are being made, you can check the sentences that differ
from the gold standard (see the data in ```data/deppars/english_test.conll```) and visualize those sentences, e.g., 
in http://www.ark.cs.cmu.edu/treeviz/.


6) (Optional.) Implement Eisner’s algorithm for projective dependency parsing. The pseudo-code is shown as Algorithm 13. Implement this algorithm as the function:

    def parse_proj(self, scores):

in file dependency decoder.py. The input is a matrix of arc scores, whose dimension is(N + 1)-by-(N + 1), and whose (h, m) entry contains the score sq(h, m). 

In particular, the first row contains the scores for the arcs that depart from the root, and the first column’s values, along with the main diagonal, are to be ignored (since no arcs point to the root, and there are no self-pointing arcs). To make your job easier, we provide an implementation of the backtracking part:

    def backtrack_eisner(self, incomplete_backtrack, complete_backtrack, s, t, direction, complete, heads):
    
so you just need to build complete/incomplete spans and their backtrack pointers and then call.

    heads = -np.ones(N+1, dtype=int) 
    self.backtrack_eisner(incomplete_backtrack, complete_backtrack, 0, N, 1, 1,heads)
    return heads

to obtain the final parse.
To test the algorithm, retrain the parser on the English data (where the trees are actually all projective) by setting
the flag dp.projective to True:

In [ ]:
dp = depp.DependencyParser() 
dp.features.use_lexical = True 
dp.features.use_distance = True 
dp.features.use_contextual = True 
dp.read_data("english") 
dp.projective = True
dp.train_perceptron(10)
dp.test()

You should get the following results:

    ￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼￼4.2.5
    Number of sentences: 8044
    Number of tokens: 80504
    Number of words: 12202
    Number of pos: 48
    Number of features: 338014
    Epoch 1
    Training accuracy: 0.835637168541
    Epoch 2
    Training accuracy: 0.922426254687
    Epoch 3
    Training accuracy: 0.947621628947
    Epoch 4
    Training accuracy: 0.960326602521
    Epoch 5
    Training accuracy: 0.967689840538
    Epoch 6
    Training accuracy: 0.97263631025
    Epoch 7
    Training accuracy: 0.97619370285
    Epoch 8
    Training accuracy: 0.979209016579
    Epoch 9
    Training accuracy: 0.98127569228
    Epoch 10
    Training accuracy: 0.981320865519
    Test accuracy (509 test instances): 0.886732599366